### 1 Bootstrapping

In [8]:
pip3 install ipykernel

SyntaxError: invalid syntax (385723413.py, line 1)

In [2]:
data = read.csv("SP 500.csv")

NameError: name 'read' is not defined

In [ ]:
data

In [ ]:
colnames(data) 

In [ ]:
price_index=data$S.P.500..Index

In [ ]:
VR = function(x,kvec)
{
x = as.matrix(x)
m = acf(x,lag.max = max(kvec),plot=FALSE)$acf[2:(max(kvec)+1)]  
VR = matrix(NA,nrow=length(kvec))
for(i in 1:length(kvec))
{
k = kvec[i]
w = (1-(1:(k-1))/k)
VR[i] = 1+ 2*sum(w*m[1:(k-1)])
}
return(VR)
}

In [ ]:
Lo_MacKinlay_stat = function(y,k)
{
    y = as.matrix(y)
    y1 = (y-mean(y))^2
    n =nrow(y) 
    
    vr = VR(y,k)
    tem1 = 2*(2*k-1)*(k-1); tem2 <- 3*k
    m1 = sqrt(n)*(vr-1)/sqrt( tem1/tem2 )
    
    w = 4*as.matrix((1-(1:(k-1))/k)^2,nrow=k-1)
    dvec = matrix(NA, nrow=(k-1), ncol=1)
    for (j in 1:(k-1))
    { 
    dvec[j] = sum(y1[(j+1):n] * y1[1:(n-j)])/( sum(y1)^2 )
    }
    summ = crossprod(w,dvec)
    m2 = sqrt(n)*(vr-1)*((n*summ)^(-.5) )
return(list(LM1=m1,LM2=m2))
}

In [ ]:
LM_stat_vec = function(y,kvec)
{
    y = as.matrix(y)
    n = nrow(y)
    mq = matrix(NA, nrow=length(kvec), ncol=2)
    for (i in 1:length(kvec))
    {
    k = kvec[i]
    LM = Lo_MacKinlay_stat(y,k)
    mq[i,] = cbind(LM$LM1,LM$LM2)
    }
    
return(list(M1=mq[,1],M2=mq[,2]))
}

In [ ]:
Bootstrap = function(r,k,B,prob=c(0.025,0.975))
{
    set.seed(666)
    r = as.matrix(r)
    VR = LM_stat_vec(r,kvec)
    
    VR_mat = matrix(NA, nrow=B, ncol=length(k))
    
    VR_stat = matrix(c(VR$M2))
    for (i in 1:B)
    {
    rs <- r * rnorm(nrow(r))
    VRs <- LM_stat_vec(rs,k)
    VR_mat[i,] <- c(VRs$M2)
    }
    
    p <- matrix(NA,nrow = ncol(VR_mat), ncol=1)
    CI <- matrix(NA,nrow = ncol(VR_mat), ncol=length(prob))
    for (i in 1:ncol(VR_mat))
        {
        tem <- abs(VR_mat[,i]) > abs(VR_stat[i])
        tem[tem == "TRUE"] <- 1
        p[i] <- mean(tem) 
        CI[i,] <- quantile(VR_mat[,i],prob)
        }
   CI <- CI[1:length(kvec),]
   colnames(CI) <- paste(100*prob,"%",sep="")
   rownames(CI) <- paste("k=",kvec,sep="")
         
return(list(Holding.Period=k,VR_statistic=as.numeric(p[1:length(k)]),CI=CI))
}

In [ ]:
nob = length(price_index)
r = log(price_index[2:nob])-log(price_index[1:(nob-1)])    
k = c(2,4,8) 
B = 1000

In [ ]:
Bootstrap(r,k,1000,prob=c(0.025,0.975))

The corresponding variance ratio computed by bootstrapping is shown in the above table.

### 2 computed by usual asymptotic distribution

In [ ]:
LM_stat_vec(r,k)

M1 is the statistic under iid homoskedastic null hypothesis, M2 is the statistic under heteroskedastic null hypothesis. So, we will use M2 statistics here and compare.

In [ ]:
P_value = function(cdf,x,paramet,side=0){
  n = length(paramet)
  p = switch(n+1,
             cdf(x),
             cdf(x,paramet),
             cdf(x,paramet[1],paramet[2]),
             cdf(x,paramet[1],paramet[2],paramet[3])
  )
  if(side <0) p
  else if(side >0) 1-p
  else
    if(p<1/2) 2*p
  else 2*(1-p)
}
P_value(pnorm,1.96,c(0,1),0)

This is the p value using the usual asymptotic distribution

In [ ]:
asp = LM_stat_vec(r,k)$M2
p_list_asp =list()
for (i in asp){
    p_list_asp= append(p_list_asp,P_value(pnorm,i,c(0,1),0))
} 
p_list_asp

This is the p value using bootstrapping

In [ ]:
bs = Bootstrap(r,k,1000,prob=c(0.025,0.975))$VR_statistic
p_list_bs =list()
for (i in bs){
    p_list_bs= append(p_list_bs,P_value(pnorm,i,c(0,1),0))
} 
p_list_bs

their p value are not the same. But the conclusion is the same, meaning that p values are larger than 0.05, rejecting the null hypothesis, which shows that the sp500 return series is not stationary and random walk. When q is small, the p value computed by two methods are quite different. However, when q becomes large, their p values are very close(0,553 versus 0.578)

### 3

There are some issues that can complicate the calculations. One particular source is the systematical financial crisis or extreme political events that brings extreme effects to financial system. For example, like period in 2008 financial crisis, the function of financial market had been collapsed, thus the market could not be efficient. Thus, when we do computations, we should get rid of those periods and use reasonable data to do furthur tests. 